In [5]:
from google.colab import drive

import functools
import os

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

tf.enable_eager_execution()

drive.mount('/content/drive', force_remount=False)
%cd /content/drive/My\ Drive/ETH/Projects/Google/source
!ls
from constants import Constants as C
from dataset import TFRecordDiagramDataset
TF_DATA_PATH = "/content/drive/My Drive/ETH/Projects/Google/data/diagrams/test/diagrams_ramer_0.025.tf_record-?????-of-?????"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ETH/Projects/Google/source
config.py     dataset_strokes.py   model_utils.py   training_eager.py
constants.py  model_components.py  None0000000.png  training_static.py
dataset.py    model.py		   __pycache__	    utils.py


In [6]:
batch_size = 8
train_data = TFRecordDiagramDataset(data_path=TF_DATA_PATH, meta_data_path=None, batch_size=batch_size, preprocessing=False, normalize=False, shuffle=False)
data_iter = train_data.get_iterator()
input_batch, target_batch = next(data_iter)

/content/drive/My Drive/ETH/Projects/Google/data/diagrams/test/diagrams_ramer_0.025.tf_record-?????-of-?????
Meta-data not found.
Skipping statistics...


In [0]:
sample_idx = 3
seq_len = input_batch[C.BATCH_SEQ_LEN][sample_idx].numpy()
vis_sample = input_batch["encoder_inputs"][sample_idx, 0:seq_len].numpy()
vis_sample[:, 1] = -1*vis_sample[:, 1]
pen = vis_sample[:, 2]
penups = np.where(pen == 1)[0]+1
strokes = np.split(vis_sample, penups, axis=0)
all_strokes = vis_sample

In [9]:
all_strokes[0]

array([[ 0.0000000e+00, -6.3520908e-01,  0.0000000e+00],
       [ 5.4633932e-04, -5.8963954e-01,  0.0000000e+00],
       [ 9.2234150e-02, -5.6556857e-01,  0.0000000e+00],
       [ 1.5938304e-01, -5.6609035e-01,  0.0000000e+00],
       [ 1.9419716e-01, -5.6212145e-01,  0.0000000e+00],
       [ 2.6904386e-01, -5.8496010e-01,  0.0000000e+00],
       [ 2.8246969e-01, -5.9282452e-01,  0.0000000e+00],
       [ 2.8553215e-01, -6.0127693e-01,  0.0000000e+00],
       [ 2.7725127e-01, -6.3689953e-01,  0.0000000e+00],
       [ 2.2491983e-01, -6.5480882e-01,  0.0000000e+00],
       [ 2.0304155e-01, -6.6661769e-01,  0.0000000e+00],
       [ 7.1321175e-02, -6.6370225e-01,  0.0000000e+00],
       [ 1.9827634e-02, -6.4397997e-01,  0.0000000e+00],
       [ 7.8497101e-03, -6.3638508e-01,  0.0000000e+00],
       [ 7.5777662e-03, -6.3104409e-01,  1.0000000e+00]], dtype=float32)

In [14]:
# First set up the figure, the axes, and the plot element
fig, ax = plt.subplots()
plt.close()
ax.set_xlim((all_strokes[:, 0].min(), all_strokes[:, 0].max()))
ax.set_ylim((all_strokes[:, 1].min(), all_strokes[:, 1].max()))
ax.set_facecolor((1.0, 1.0, 1.0))

lines = []
line_borders = []  # Start and end index of a stroke in the entire drawing.
current_len = 0
for index, stroke in enumerate(strokes):
    line_borders.append((current_len, current_len+stroke.shape[0]))
    lines.append(ax.plot([],[],lw=2, color=None)[0])
    current_len += stroke.shape[0] + 1

def init():
    for line in lines:
        line.set_data([],[])
    return lines
  
# animation function
def animate(i):
  for idx, (line, (line_start, line_end)) in enumerate(zip(lines, line_borders)):
    if i > line_end:
      # All strokes that have been visualized.
      line.set_data(strokes[idx][:, 0], strokes[idx][:, 1])
    else:
      line.set_data(strokes[idx][0:(i-line_start), 0], strokes[idx][0:(i-line_start), 1])
      break
  return lines

anim = animation.FuncAnimation(fig, animate, init_func=init, frames=all_strokes.shape[0], interval=100, blit=True)
rc('animation', html='jshtml', embed_limit=128)
anim

# from IPython.display import HTML
# HTML(anim.to_html5_video(embed_limit=128))

Output hidden; open in https://colab.research.google.com to view.